In [ ]:
import os
import pandas as pd
import numpy as np

from pipeline.backend.pipeline import PipeLine
from pipeline.component.dataio import DataIO
from pipeline.component.homo_lr import HomoLR
from pipeline.component.reader import Reader
from pipeline.interface.data import Data
from pipeline.interface.model import Model
from pipeline.component.evaluation import Evaluation
from pipeline.component.scale import FeatureScale
from pipeline.utils.tools import load_job_config
from pipeline.runtime.entity import JobParameters

from sklearn.preprocessing import RobustScaler


In [ ]:
# simple feature engineering
df = pd.read_csv('/Examples/Pipeline/notebooks/creditcard_host.csv')

robust_scale = RobustScaler()
df['Scaled_Amount'] = robust_scale.fit_transform(df['Amount'].values.reshape(-1,1))
df['Scaled_Time'] = robust_scale.fit_transform(df['Time'].values.reshape(-1,1))
IDs = [i + 1 for i in range(len(df))]

Scaled_Amount = df['Scaled_Amount']
Scaled_Time = df['Scaled_Time']
df.drop(['Scaled_Amount','Scaled_Time'], axis=1, inplace=True)
df.insert(0, 'id', IDs)
df.insert(1, 'Scaled_Amount', Scaled_Amount)
df.insert(2, 'Scaled_Time', Scaled_Time)

df.drop(['Time','Amount'], axis=1, inplace=True)

df.to_csv('/Examples/Pipeline/notebooks/creditcard_host_scaled.csv', index=False)

In [ ]:
guest = 10000

# spark + pulsar
backend = 2
work_mode = 1

partition = 8

host_train_data = {"name": "dataset_host", "namespace": f"experiment"}

host_eval_data = {"name": "dataset_host", "namespace": f"experiment"}

pipeline_upload = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest)
# add upload data info
# original csv file path
pipeline_upload.add_upload_data(file=os.path.join('/Examples/Pipeline/notebooks/creditcard_host_scaled.csv'),
                                table_name=host_train_data["name"],
                                namespace=host_train_data["namespace"],
                                head=1, partition=partition)
# upload all data
pipeline_upload.upload(work_mode=work_mode, backend=backend, drop=1)